In [ ]:
from util.addpath import addpath
addpath("../code")

In [ ]:
from statsmodels.stats.weightstats import DescrStatsW
from patsy import dmatrices

## March-CPS

### march-cleaners

### cleaned-data  

In [ ]:
input_path = "../../ref/origin/March-CPS/cleaned-data/"

In [ ]:
#%ls $input_path

#### variables

In [ ]:
def show_var_label(y=1999): 
    data = pd.read_stata(input_path+f"mar{str(y)[-2:]}.dta", iterator = True)
    labels = data.variable_labels()
    print(len(labels))
    return labels

In [ ]:
show_var_label(1966)

### prep_wage

In [ ]:
input_path = "../../ref/origin/March-CPS/cleaned-data/"

#### tab-march-ineq

In [ ]:
df = pd.read_stata(input_path+f"mar{str(1976)[-2:]}.dta")

In [ ]:
df.exp

In [ ]:
bins = range(0,51,10)
pd.cut(df.exp,bins,right=False,labels=range(5,46,10))

In [ ]:
from MarchCPS.prep_wage import tabulate_march_inequality

In [ ]:
a,b = tabulate_march_inequality(1992)

#### tab-march-ineq-loop

In [ ]:
from MarchCPS.prep_wage import tabulate_march_inequality_loop

In [ ]:
ineq_stat = tabulate_march_inequality_loop()

In [ ]:
ineq_stat

#### predict_archwg_regs_exp

In [ ]:
from MarchCPS.prep_wage import tabulate_march_basic

In [ ]:
df = tabulate_march_basic(1976)

In [ ]:

    df = df.eval("""
            lnwinc = log(winc_ws)
            lnhinc = log(hinc_ws)
        """)

    # Create experience categories
    # @ the code split exp by [0,9] [10,19], ... however for years >= 1992 there can be 9.x 19.x in exp
    # @ here I split by [0,10), [10,20), ... thus the result might be differnt from the original one
    df = df.rename(columns={"exp": "exp1"})
    assert df.eval("0<= exp1 <=48").all()
    bins = range(0, 51, 10)
    # @ rather than construct dummy columns, here set category column
    df = df.assign(expcat5=pd.cut(df.exp1, bins, right=False, labels=range(5, 46, 10)))

    # experience education interactions;
    def prod_exp_edu_interactions(dx):
        dx = dx.eval("""
                exp2 = (exp1**2)/100
                exp3 = (exp1**3)/1000
                exp4 = (exp1**4)/10000

                e1edhsd = exp1*edhsd
                e1edsmc = exp1*edsmc
                e1edclg = exp1*(edclg+edgtc)

                e2edhsd = exp2*edhsd
                e2edsmc = exp2*edsmc
                e2edclg = exp2*(edclg+edgtc)

                e3edhsd = exp3*edhsd
                e3edsmc = exp3*edsmc
                e3edclg = exp3*(edclg+edgtc)

                e4edhsd = exp4*edhsd
                e4edsmc = exp4*edsmc
                e4edclg = exp4*(edclg+edgtc)
        """)
        return dx
    
    df = prod_exp_edu_interactions(df)

    # Gender interactions with experience, race, region, part-time;
    df = df.eval("""
            pt = abs(fulltime-1)

            fexp1 = female*exp1
            fexp2 = female*exp2
            fexp3 = female*exp3
            fexp4 = female*exp4

            fblack = female*black
            fother = female*other

            fpt = female*pt
    """)

    # Full-time samples
    df = df.eval("ftfy = fulltime*fullyear")

    #@ I skip some summ code here

    # we don't want to drop top-coded obs
    #@ no idea what this is
    df.tcwkwg = 0
    df.tchrwg = 0

In [ ]:
    # Male regression
    dt = df.query("ftfy==1 & female==0 & tcwkwg==0 & bcwkwgkm==0")
    X_ = ["edhsd", "edsmc", "edclg", "edgtc",
        "exp1", "exp2", "exp3", "exp4",
        "e1edhsd", "e1edsmc", "e1edclg",
        "e2edhsd", "e2edsmc", "e2edclg",
        "e3edhsd", "e3edsmc", "e3edclg",
        "e4edhsd", "e4edsmc", "e4edclg",
        "black", "other"]
    y, X = dt.lnwinc, dt[X_]
    X = sm.add_constant(X)
    res = sm.WLS(y, X, weights=dt.wgt).fit()

    # Predict values
    # Now conduct 25 predictions based upon the the 5 education and 5 experience categories
    edcat = range(5)
    edcol = ["edhsd", "edhsg", "edsmc", "edclg", "edgtc"]
    expcat = range(5, 46, 10)
    expcol = [f"exp{i}" for i in expcat]
    s = pd.DataFrame(0, index=pd.MultiIndex.from_product([edcol, expcol]), columns=X_)
    for ed in edcat:
        n = edcol[ed]
        if n != "edhsg":
            s.loc[n, n] = 1
    s = s.swaplevel()
    for i, exp in enumerate(expcat):
        n = expcol[i]
        s.loc[n, "exp1"] = exp
    X_cf = sm.add_constant(prod_exp_edu_interactions(s))
    marchwk_ed_exp_m = res.predict(X_cf)  # @ tidy dataframe for marchwk-`ed'-exp`exp'-m

    # Female regression
    dt = df.query("ftfy==1 & female==1 & tcwkwg==0 & bcwkwgkm==0")
    y, X = dt.lnwinc, dt[X_]
    X = sm.add_constant(X)
    res = sm.WLS(y, X, weights=dt.wgt).fit()

    # Predict values
    # Now conduct 25 predictions based upon the the 5 education and 5 experience categories
    marchwk_ed_exp_f = res.predict(X_cf)  # @ tidy dataframe for marchwk-`ed'-exp`exp'-f

    # Compile all male/female predicted weekly wages
    marchwk_ed_exp = s.assign(plnwkw_m=marchwk_ed_exp_m, plnwkw_f=marchwk_ed_exp_f)

In [ ]:
marchwk_ed_exp